In [ ]:
# Importing programs, porting in database
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [ ]:
# Creating list of home teams, away teams, and winners
c.execute("""select date, HomeTeam, AwayTeam, FTHG, FTAG, FTR
                    from matches
                    where Season == 2011;""")
df_res = pd.DataFrame(c.fetchall())
df_res.columns = [x[0] for x in c.description]

In [ ]:
# Changing date to datetime
df_res['Date'] = df_res.Date.map(lambda x:
                                 pd.to_datetime(x[:10],
                                                format='%Y/%m/%d'))

In [ ]:
# Creating a list of winning and losing teams for each game
df_res['homeValue'] = df_res.apply((lambda x: "win" if x['FTR'] == "H" else (
    "loss" if x['FTR'] == "A" else "draw")), axis=1)

df_res['awayValue'] = df_res.apply((lambda x: "win" if x['FTR'] == "A" else (
    "loss" if x['FTR'] == "H" else "draw")), axis=1)

In [ ]:
# Creating lists of winning teams
df_win_home = df_res.loc[df_res.FTR == "H", ['Date', 'HomeTeam', 'FTHG']]
df_win_away = df_res.loc[df_res.FTR == "A", ['Date', 'AwayTeam', 'FTAG']]

# Unifying column names
df_win_away = df_win_away.rename(columns={'AwayTeam': 'Team', "FTAG": "Goals"})
df_win_home = df_win_home.rename(columns={'HomeTeam': 'Team', "FTHG": "Goals"})

# Creating full list of winning teams
df_win = pd.concat([df_win_home, df_win_away])

In [ ]:
# Creating lists of losing teams
df_lose_home = df_res.loc[df_res.FTR == "A", ['Date', 'HomeTeam', 'FTHG']]
df_lose_away = df_res.loc[df_res.FTR == "H", ['Date', 'AwayTeam', 'FTAG']]

In [ ]:
# Unifying column names
df_lose_away = df_lose_away.rename(
    columns={'AwayTeam': 'Team', "FTAG": "Goals"})
df_lose_home = df_lose_home.rename(
    columns={'HomeTeam': 'Team', "FTHG": "Goals"})

In [ ]:
# Creating full list of losing teams
df_lose = pd.concat([df_lose_home, df_lose_away])

In [ ]:
def winloss(team):
    wins = df_win.loc[(df_win['Team'] == team), ['Goals']]
    losses = df_lose.loc[(df_lose['Team'] == team), ['Goals']]
    losses = list(losses.Goals)
    wins = list(wins.Goals)
    plt.figure(figsize=(10, 8))
    plt.hist(x=wins, bins=(max(wins)+1), density=True, alpha=.5, color='blue')
    plt.hist(x=losses, bins=(max(wins)+1),
             density=True, alpha=.5, color='green')
    plt.xticks(range(max(wins)+1))
    print(list(wins))
    print(list(losses))